### Initial exploration of data features
In this notebook we will analyse some of the initial data, by finding out what features are available, what granularity the data comes in, as well as seeing if there is any obvious noise we need to take into account.

In [1]:
experiment = "data/experiments/experiment_2/"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
## Loading in the data from data/experiments/experiment_1, which contains button_presses.csv for the labels, and other csv files for the features. In experiments_1/meta we have the system time and the device info
def load_data(path, within_range=True, temp_features=True):
    # Load the starting time
    time_df = pd.read_csv(path + 'meta/time.csv')
    start_time = time_df.loc[time_df['event'] == 'START', 'system time'].iloc[0]

    data_frames = []
    for filename in os.listdir(path):
        if filename.endswith(".csv"):
            df = pd.read_csv(path + filename)
            # Check if 'Time (s)' column exists
            if 'Time (s)' in df.columns:
                # Convert 'Time (s)' column to datetime index for each dataframe
                df.index = pd.to_datetime(df['Time (s)'], unit='s', origin=pd.Timestamp(start_time, unit='s'))
                data_frames.append(df)
            else:
                print(f"'Time (s)' column not found in file: {filename}")
                print(f"Columns found: {df.columns}")
    
    # Concatenate dataframes
    data = pd.concat(data_frames)
    
    # resample to 10 Hz
    data_resampled = data.resample('100ms').mean()
    
    # Load label dataset
    labels = pd.read_csv(path+'button_presses.csv', names=['Timestamp', 'Label'])
    labels['Timestamp'] = pd.to_datetime(labels['Timestamp'], unit='s')
    
    # Filter timestamps within label range
    if within_range:
        first_label_timestamp = labels['Timestamp'].iloc[0]
        last_label_timestamp = labels['Timestamp'].iloc[-1]
        data_resampled = data_resampled[(data_resampled.index >= first_label_timestamp) & (data_resampled.index <= last_label_timestamp)]
    
    if len(data_resampled):
        # Add labels
        def get_recent_label(row):
            return labels[labels['Timestamp'] <= row.name]['Label'].iloc[-1]

        data_resampled['Label'] = data_resampled.apply(get_recent_label, axis=1)

        # Add temporal label features
        def get_time_until_next(row):
            next_label = labels[labels['Timestamp'] > row.name]['Timestamp'].min()
            if pd.isnull(next_label):
                return pd.NaT
            else:
                return (next_label - row.name).total_seconds()

        def get_time_since_previous(row):
            previous_label = labels[labels['Timestamp'] < row.name]['Timestamp'].max()
            if pd.isnull(previous_label):
                return pd.NaT
            else:
                return (row.name - previous_label).total_seconds()

        data_resampled['Time_Until_Next_Label'] = data_resampled.apply(get_time_until_next, axis=1)
        data_resampled['Time_Since_Previous_Label'] = data_resampled.apply(get_time_since_previous, axis=1)
        data_resampled['Time_Until_Next_Label'] = data_resampled['Time_Until_Next_Label'].fillna(0.0)
        data_resampled['Time_Since_Previous_Label'] = data_resampled['Time_Since_Previous_Label'].fillna(0.0)
    return data_resampled

data = load_data(experiment)
# display(data)

In [ ]:
import xml.etree.ElementTree as ET
from datetime import datetime

# Define a conversion function
def convert_timestamp(timestamp):
    datetime_obj = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%S.%fZ')
    return datetime_obj

# Load the xml file into a dataframe
def load_xml(path, convert_time=True):
    # Parse the XML file
    tree = ET.parse(path + 'activity_11340269258.tcx')
    root = tree.getroot()

    # Define the namespaces
    namespaces = {
        'tc': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2',
        'activity': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2',
        'ns3': 'http://www.garmin.com/xmlschemas/ActivityExtension/v2',
        'ns5': 'http://www.garmin.com/xmlschemas/ActivityGoals/v1',
        'ns2': 'http://www.garmin.com/xmlschemas/UserProfile/v2',
        'xsi': 'http://www.w3.org/2001/XMLSchema-instance',
        'ns4': 'http://www.garmin.com/xmlschemas/ProfileExtension/v1'
    }

    # Extract data from XML and create a dictionary
    xml_data = {'Time': [], 'AltitudeMeters': [], 'HeartRate': []}

    for trackpoint in root.findall('.//tc:Trackpoint', namespaces):
        time = trackpoint.find('tc:Time', namespaces).text
        altitude = trackpoint.find('tc:AltitudeMeters', namespaces).text
        heart_rate = trackpoint.find('tc:HeartRateBpm/tc:Value', namespaces).text

        xml_data['Time'].append(time)
        xml_data['AltitudeMeters'].append(altitude)
        xml_data['HeartRate'].append(heart_rate)

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(xml_data)
    
    df['AltitudeMeters'] = df['AltitudeMeters'].astype(float)
    df['HeartRate'] = df['HeartRate'].astype(float)
    
    # Apply the conversion function to the 'Time' column
    if convert_time:
        df['Time'] = df['Time'].apply(convert_timestamp)
    
    df = df.set_index('Time')
    
    return df

xml_data = load_xml(experiment)

In [ ]:
# Merges the csv and xml data
def merge(data, xml_data):
    first_timestamp = data.index[0]
    last_timestamp = data.index[-1]
    df_filtered = xml_data[(xml_data.index >= first_timestamp) & (xml_data.index <= last_timestamp)]
    merged_df = pd.merge(data, df_filtered, left_index=True, right_index=True, how='left')
    
    return merged_df
df = merge(data, xml_data)


## Load dataframe from file

In [ ]:
df = pd.read_csv(experiment + 'merged/added_features.csv')

## Change labels to numbers

In [ ]:
df['Label'] = df['Label'].astype('category').cat.codes

## Remove unuseful columns

In [ ]:
def remove_columns(df, columns):
    return df.drop(columns=columns)

# 'Latitude (°)' 'Longitude (°)' 'X (hPa)' nog evt
columns = ['Velocity (m/s)', 'Direction (°)', 'Distance (cm)', 'Horizontal Accuracy (m)', 'Time_Until_Next_Label', 'Time_Since_Previous_Label', 'Time (s)']
df_filt = df.drop(columns=columns)
df_filt = df_filt.rename(columns={'Time (s).1': 'Time (s)'})

In [ ]:
df_filt

## Interpolation

In [ ]:
from ImputationMissingValues import ImputationMissingValues

# Interpolates specified columns of the dataframe (all columns with NaNs as default)
def interpolate(df, columns=None):
    if columns is None:
        columns = df.columns[df.isna().any()].tolist()
    
    imputer = ImputationMissingValues()
    for column in columns:
        df = imputer.impute_interpolate(df, column)
    
    return df

interpolated_df = interpolate(df_filt)
# interpolated_df

In [ ]:
interpolated_df

## Adding features
Mean, std, min, max, difference

In [ ]:
def create_features(df, seconds, columns=None):
    # Create a new DataFrame to store the aggregated values
    new_df = pd.DataFrame()

    # Define the window size
    window_size = seconds * 10
    
    if columns == None:
        columns = df.loc[:, ~df.columns.str.startswith('Label')].columns

    # Iterate over the rolling windows in the original DataFrame
    for i in range(len(df) - window_size + 1):
        # Select a rolling window subset
        subset = df.iloc[i:i+window_size]

        # Iterate over each column in the subset
        for col in columns:
            if col == "Time (s)":
                new_df.loc[i, f'Starttime (s)'] = subset[col].iloc[0]
                new_df.loc[i, f'Endtime (s)'] = subset[col].iloc[-1]
            
            else:
                col_mean = subset[col].mean()
                col_std = subset[col].std()
                col_min = subset[col].min()
                col_max = subset[col].max()
                col_diff = subset[col].iloc[-1] - subset[col].iloc[0]

                # Create new columns in the new DataFrame
                new_df.loc[i, f'{col} mean'] = col_mean
                new_df.loc[i, f'{col} std'] = col_std
                new_df.loc[i, f'{col} min'] = col_min
                new_df.loc[i, f'{col} max'] = col_max
                new_df.loc[i, f'{col} diff'] = col_diff

        # Get the most frequent label within the window
        most_frequent_label = subset['Label'].mode().iloc[0]
        new_df.loc[i, 'Label'] = most_frequent_label

    # Reset the index of the new DataFrame
    new_df.reset_index(drop=True, inplace=True)

    # Print the new DataFrame
    return new_df


In [3]:
df_full = pd.read_csv(experiment + 'merged/added_features_stats.csv')

## Principal Component Analysis

In [4]:
from DataTransformation import PrincipalComponentAnalysis

# PCA function where number of PC's and columns used can be specified (all columns but the label as default).
# Adds new PCA columns to the dataframe
def pca(df, num_comp, columns=None):
    if columns is None:
        columns_to_exclude = ['Starttime (s)', 'Endtime (s)', 'Label']
        columns = [col for col in df.columns if col not in columns_to_exclude]
    PCA = PrincipalComponentAnalysis()
    pca_df = PCA.apply_pca(df, columns, num_comp)
    
    return pca_df

def determine_pc_explained_variance(df, columns = None):
    if columns is None:
        columns_to_exclude = ['Starttime (s)', 'Endtime (s)', 'Label']
        columns = [col for col in df.columns if col not in columns_to_exclude]
    PCA = PrincipalComponentAnalysis()
    ratio = PCA.determine_pc_explained_variance(df, columns)
    
    return ratio

ratio = determine_pc_explained_variance(df_full, columns = None)

In [5]:
cumulative_variance = np.cumsum(ratio)
n_components = np.argmax(cumulative_variance >= 0.95) + 1

print("Number of components to keep:", n_components)

Number of components to keep: 27


In [126]:
pca_df = pca(df_full, n_components)
pca_df

,Starttime (s),Endtime (s),X (m/s^2) mean,X (m/s^2) std,X (m/s^2) min,X (m/s^2) max,X (m/s^2) diff,Y (m/s^2) mean,Y (m/s^2) std,Y (m/s^2) min,Y (m/s^2) max,Y (m/s^2) diff,Z (m/s^2) mean,Z (m/s^2) std,Z (m/s^2) min,Z (m/s^2) max,Z (m/s^2) diff,X (hPa) mean,X (hPa) std,X (hPa) min,X (hPa) max,X (hPa) diff,X (rad/s) mean,X (rad/s) std,X (rad/s) min,X (rad/s) max,X (rad/s) diff,Y (rad/s) mean,Y (rad/s) std,Y (rad/s) min,Y (rad/s) max,Y (rad/s) diff,Z (rad/s) mean,Z (rad/s) std,Z (rad/s) min,Z (rad/s) max,Z (rad/s) diff,Latitude (°) mean,Latitude (°) std,Latitude (°) min,Latitude (°) max,Latitude (°) diff,Longitude (°) mean,Longitude (°) std,Longitude (°) min,Longitude (°) max,Longitude (°) diff,Height (m) mean,Height (m) std,Height (m) min,Height (m) max,Height (m) diff,Vertical Accuracy (°) mean,Vertical Accuracy (°) std,Vertical Accuracy (°) min,Vertical Accuracy (°) max,Vertical Accuracy (°) diff,X (µT) mean,X (µT) std,X (µT) min,X (µT) max,X (µT) diff,Y (µT) mean,Y (µT) std,Y (µT) min,Y (µT) max,Y (µT) diff,Z (µT) mean,Z (µT) std,Z (µT) min,Z (µT) max,Z (µT) diff,AltitudeMeters mean,AltitudeMeters std,AltitudeMeters min,AltitudeMeters max,AltitudeMeters diff,HeartRate mean,HeartRate std,HeartRate min,HeartRate max,HeartRate diff,Roll (°) mean,Roll (°) std,Roll (°) min,Roll (°) max,Roll (°) diff,Pitch (°) mean,Pitch (°) std,Pitch (°) min,Pitch (°) max,Pitch (°) diff,Yaw (°) mean,Yaw (°) std,Yaw (°) min,Yaw (°) max,Yaw (°) diff,Label,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,pca_20,pca_21,pca_22,pca_23,pca_24,pca_25,pca_26,pca_27
0,4.049522,8.947556,4.668066,0.736433,2.254366,6.540396,0.564754,0.247722,0.687264,-0.974138,2.154488,-0.039186,0.943490,1.293481,-1.257709,4.701795,-2.077609,1015.844628,0.001750,1015.843048,1015.849823,0.004105,-0.018197,0.288434,-0.999210,0.578402,0.112204,-0.003596,0.484012,-1.052395,1.348128,-0.220877,-0.020165,0.166541,-0.384679,0.233272,0.041534,52.355728,0.000001,52.355727,52.355731,0.000004,4.956966,2.195755e-07,4.956965,4.956966,-7.472308e-07,-2.208062,0.006206,-2.225801,-2.204683,-0.021119,43.221538,0.406872,43.000000,44.384615,1.384615,-38.165336,4.596663,-42.750142,-25.614095,-3.855437,-12.642756,1.889757,-15.835959,-8.784474,3.040558,-13.104054,7.943040,-32.317590,-5.507129,2.099371,-33.656726,0.047532,-33.741817,-33.599998,-0.141819,82.031818,0.334935,81.0,82.354545,1.354545,-74.049133,11.886516,-88.847026,-46.183610,-25.978217,9.767464,61.738170,-174.320612,131.182650,0.558601,-8.402301,4.001984,-14.794028,0.188840,-5.700601,0.0,0.265961,0.314463,0.338350,0.367011,0.307676,-0.168426,-0.813453,0.333065,0.318347,0.091337,0.199322,0.056509,0.179423,-0.110070,0.088244,-0.030301,-0.221104,-0.056230,0.003013,0.013705,-0.294196,-0.258365,0.103761,-0.246610,-0.006830,0.030016,0.054190
1,4.149481,9.047517,4.681272,0.732677,2.254366,6.540396,-1.740343,0.248589,0.687040,-0.974138,2.154488,-1.025743,0.899289,1.276461,-1.257709,4.701795,-0.831744,1015.844722,0.001926,1015.843048,1015.850410,0.004692,-0.017758,0.288501,-0.999210,0.578402,0.179526,-0.009020,0.482772,-1.052395,1.348128,-0.195523,-0.019171,0.166738,-0.384679,0.233272,0.004649,52.355728,0.000001,52.355727,52.355731,0.000004,4.956966,2.388087e-07,4.956965,4.956966,-7.970462e-07,-2.208512,0.006749,-2.227209,-2.204683,-0.022527,43.251077,0.442511,43.000000,44.476923,1.476923,-38.243963,4.641571,-42.755756,-25.614095,-4.027117,-12.579805,1.860284,-15.835959,-8.784474,2.930623,-13.060916,7.979281,-32.317590,-5.449609,2.750525,-33.659635,0.048432,-33.745454,-33.599998,-0.145455,82.059091,0.303220,81.1,82.363636,1.263636,-74.598732,11.799771,-88.847026,-46.183610,-9.930988,10.096522,61.735405,-174.320612,131.182650,-27.060404,-8.512004,3.838812,-14.794028,0.188840,-5.673973,0.0,0.265705,0.312726,0.339895,0.366083,0.304637,-0.171658,-0.811701,0.332987,0.319077,0.093688,0.199153,0.062712,0.181236,-0.106040,0.073863,-0.039326,-0.214258,-0.067937,-0.031952,0.042633,-0.

## Support Vector Machine

In [120]:
from LearningAlgorithms import ClassificationAlgorithms

X = pca_df.iloc[:, -num_comp:]
Y = pca_df.loc[:, pca_df.columns.str.startswith('Label')]
split = int(0.8 * len(pca_df))

clf = ClassificationAlgorithms()
pred_training_y, pred_test_y, frame_prob_training_y, frame_prob_test_y = clf.support_vector_machine_with_kernel(X[:split], Y[:split], X[split:])

KeyboardInterrupt: 

## Plotting

In [ ]:
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from statsmodels.tsa.seasonal import seasonal_decompose

def plot_data(data, output_path):
    # Set Seaborn style and context
    sns.set_style("whitegrid")
    sns.set_context("paper")

    # Line plot
    fig, ax = plt.subplots()
    sns.lineplot(data=data, palette="tab10", linewidth=1.5, ax=ax)
    ax.set_title('Time Series Line Plot')
    ax.set_xlabel('Time')
    ax.set_ylabel('Values')
    fig.savefig(output_path + 'line_plot.pdf')

    # Distribution of the data
    fig, ax = plt.subplots()
    sns.kdeplot(data=data, ax=ax, fill=True)
    ax.set_title('Data Distribution Plot')
    ax.set_xlabel('Values')
    fig.savefig(output_path + 'distribution_plot.pdf')

    # Boxplots of the data
    fig, ax = plt.subplots()
    sns.boxplot(data=data, palette="tab10", ax=ax)
    ax.set_title('Data Boxplot')
    fig.savefig(output_path + 'boxplot.pdf')

    # Correlation matrix of the data
    fig, ax = plt.subplots()
    sns.heatmap(data.corr(), annot=True, cmap='coolwarm', ax=ax)
    ax.set_title('Data Correlation Matrix')
    fig.savefig(output_path + 'correlation_matrix.pdf')

    # Pairwise relationships
    fig, ax = plt.subplots()
    sns.pairplot(data)
    ax.set_title('Pairwise Relationships')
    fig.savefig(output_path + 'pairwise_relationships.pdf')

    # Histogram
    fig, ax = plt.subplots()
    data.hist(bins=30, ax=ax)
    ax.set_title('Data Histogram')
    fig.savefig(output_path + 'histogram.pdf')

    # Time series decomposition
    for col in data.columns:
        try:
            result = seasonal_decompose(data[col], model='additive', period=1)
            fig, (ax1,ax2,ax3,ax4) = plt.subplots(4,1, figsize=(10,8))
            result.observed.plot(ax=ax1)
            ax1.set_ylabel('Observed')
            result.trend.plot(ax=ax2)
            ax2.set_ylabel('Trend')
            result.seasonal.plot(ax=ax3)
            ax3.set_ylabel('Seasonal')
            result.resid.plot(ax=ax4)
            ax4.set_ylabel('Residual')
            fig.savefig(output_path + f'{col}_decomposition.pdf')
        except:
            print(f"Cannot decompose {col}")

output_path = experiment + "figures/"
plot_data(df, output_path)